# Imports

In [5]:
!pip install chart-studio
import pandas as pd
import numpy as np
import calendar
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
!pip install prophet
# Facebook's Prophet forecasting library
from prophet import Prophet

# Standard plotly imports
import chart_studio.plotly.plotly as py
from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Pulling the data

In [6]:
# Read in the raw temperature and emissions datasets (they are in CSV format)
raw_t = pd.read_csv('./GLB.Ts+dSST.csv', skiprows=1)
raw_e = pd.read_csv('./API_EN.ATM.CO2E.PC_DS2_en_csv_v2_10515380.csv', skiprows=3)

FileNotFoundError: ignored

In [ ]:
# Investigate how the raw data looks
raw_t.head()

In [ ]:
raw_e.head()

# Wrangling Time Series Data

## Wrangling Temperature

In [ ]:
# Create new dataframe with an index for each month
# First create the date range
date_rng = pd.date_range(start='1/1/1880', end='1/03/2019', freq='M')

type(date_rng[0])
### returns
### pandas._libs.tslibs.timestamps.Timestamp

# Next create the empty DataFrame, which we will populate using the actual data
t = pd.DataFrame(date_rng, columns=['date'])

# Create a column for the anomoly values
t['Avg_Anomaly_deg_C'] = None

# Set the index of the DataFrame to the date column (DateTime index)
t.set_index('date', inplace=True)

# Show the first few elements
t.head()

In [ ]:
# We only want the monthly data, lets only select that and leave out the seasonal columns
raw_t = raw_t.iloc[:,:13]
raw_t.head()

In [ ]:
# Import relevant libraries
import calendar
from datetime import datetime

# Function definition
def populate_df_with_anomolies_from_row(row):
    year = row['Year']
    # Anomaly values (they seem to be a mixture of strings and floats)
    monthly_anomolies = row.iloc[1:]
    # Abbreviated month names (index names)
    months = monthly_anomolies.index
    for month in monthly_anomolies.index:
        # Get the last day for each month
        last_day = calendar.monthrange(year,datetime.strptime(month, '%b').month)[1]
        # construct the index with which we can reference our new DataFrame (to populate)
        date_index = datetime.strptime(f'{year} {month} {last_day}', '%Y %b %d')
        # Populate / set value @ above index, to anomaly value
        t.loc[date_index] = monthly_anomolies[month]

# Apply function to each row of raw data
_ = raw_t.apply(lambda row: populate_df_with_anomolies_from_row(row), axis=1)

# Show the first few elements of our newly populated DataFrame
t.head()

In [ ]:
# Import Numpy, as library meant for large arrays - we will use it that we
import numpy as np

# Define function to convert values to floats, and return a 'NaN = Not a Number' if this is not possible
def clean_anomaly_value(raw_value):
    try:
        return float(raw_value)
    except:
        return np.NaN

# Apply above function to all anomaly values in DataFrame
t['Avg_Anomaly_deg_C'] = t['Avg_Anomaly_deg_C'].apply(lambda raw_value: clean_anomaly_value(raw_value))

# 'Forward fill' to take care of NaN values
t.fillna(method='ffill', inplace=True)

# Show the first few elements of our newly cleaned DataFrame
t.head()

In [ ]:
# Plot the data - quick and easy - using matplotlib, we will draw prettier graphs later

# Import Matplotlib
import matplotlib.pyplot as plt
# Allow for rendering within notebook
%matplotlib inline

# Create figure, title and plot data
plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('Temperature Anomaly (°Celsius)')
plt.plot(t, color='#1C7C54', linewidth=1.0)

In [ ]:
# The data can be resampled to a different frequency,
# lets resample it to years (end-of-year):
t.resample('A').mean().head()

In [ ]:
# Now lets visualize our resampled DataFrame

# Import Matplotlib
import matplotlib.pyplot as plt
# Allow for rendering within notebook
%matplotlib inline
# Create figure, title and plot resampled data
plt.figure(figsize=(10,8))
plt.xlabel('Time')
plt.ylabel('Temperature Anomaly (°Celsius)')
plt.plot(t.resample('A').mean(), color='#1C7C54', linewidth=1.0)

## Wrangling Emissions

In [ ]:
# Define function to pull value from raw data, using DateIndex from new DataFrame row
def populate_df(row):
    index = str(row['date'].year)
    value = raw_e_world.loc[index]
    return value

# Select just the co2 emissions for the 'world', and the columns for the years 1960-2018
raw_e_world = raw_e[raw_e['Country Name']=='World'].loc[:,'1960':'2018']

# 'Traspose' the resulting slice, making the columns become rows and vice versa
raw_e_world = raw_e_world.T
raw_e_world.columns = ['value']

# Create a new DataFrame with a daterange the same the range for..
# the Temperature data (after resampling to years)
date_rng = pd.date_range(start='31/12/1960', end='31/12/2018', freq='y')
e = pd.DataFrame(date_rng, columns=['date'])

# Populate the new DataFrame using the values from the raw data slice
v = e.apply(lambda row: populate_df(row), axis=1)
e['Global CO2 Emissions per Capita'] = v
e.set_index('date', inplace=True)
e.head()

In [ ]:
# Illustrate how the above DataFrame can be filtered for values after 2011
e[e.index.year>2011]

In [ ]:
# Deal with the NaNs by 'forward filling'
e.fillna(method='ffill', inplace=True)

In [ ]:
# Check that there are no longer NaNs at the end
e[e.index.year>2011]

In [ ]:
e['1984-01-04':'1990-01-06']

# Final DataFrames

In [ ]:
# Temperature anomalies
t.head()

In [ ]:
# Temperature anomalies resampled to end-of-year
t.resample('A').mean().head()

In [ ]:
# CO2 emissions
e.head()

# Visualizing Time Series Data

## Plotting Temperature with Matplotlib

In [ ]:
# import Matplotlib
import matplotlib.pyplot as plt
# Allow for graphs to be displayed in Jupyter notebook
%matplotlib inline

# Resample or temperature data to years (end-of-year)
t_resampled = t.resample('A').mean()

# Create figures and axes
fig, ax = plt.subplots(figsize=(10,8))

# Plot temperature data with specific colour and line thickness
ax.plot(t_resampled, color='#1C7C54', linewidth=2.5)

# Set axis labels and graph title
ax.set(xlabel='Time (years)', ylabel='Temperature Anomaly (deg. Celsius)',
       title='Global Temperature Anomalies')

# Enable grid
ax.grid()


## Plotting Emissions with Matplotlib

In [ ]:
# import Matplotlib
import matplotlib.pyplot as plt
# Allow for graphs to be displayed in Jupyter notebook
%matplotlib inline

# Create figures and axes
fig, ax = plt.subplots(figsize=(10,8))

# Plot co2 emissions data with specific colour and line thickness
ax.plot(e, color='#3393FF', linewidth=2.5)

# Set axis labels and graph title
ax.set(xlabel='Time (years)', ylabel='Emissions (Metric Tons per Capita)',
       title='Global CO2 Emission over Time')

# Enable grid
ax.grid()

## Plotting Temperature with Plotly

In [ ]:
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [ ]:
t.resample('A').mean().iplot(kind='line', xTitle='Time (years)', color='#1C7C54',
                  yTitle='Temperature Anomaly (deg. Celsius)', title='Global Temperature Anomalies')

## Plotting Emissions with Plotly

In [ ]:
e.iplot(kind='line', xTitle='Time (years)', color='#3393FF',
                  yTitle='Emissions (Metric Tons per Capita)', title='Global CO2 Emission over Time')

# Timeseries Correlation

## Granger Causality

## Dynamic Time Warping

In [ ]:
# The above will be available in another post soon! The post will also use these same two time series.

# Time Series Modelling and Forecasting

## Facebook Prophet

In [ ]:
t_prophet = pd.DataFrame()
t_prophet['ds'] = t.index
t_prophet['y'] = t['Avg_Anomaly_deg_C'].values

In [ ]:
m = Prophet()
m.fit(t_prophet)

In [ ]:
# Make future dataframes
future = m.make_future_dataframe(freq='m', periods=100*12)
forecast = m.predict(future)

In [ ]:
m.plot(forecast);

In [ ]:
m.plot_components(forecast);

## Autoregressive Integrated Moving Average (ARIMA)

In [ ]:
# This will also be available in a future post

# THE END